In [1]:
# https://cran.r-project.org/web/packages/penaltyLearning/index.html

In [2]:
# libraries
library(penaltyLearning)
library(data.table)
library(future)
library(future.apply)
library(directlabels)

In [3]:
# Get the names of all folders in a specified directory
datasets <- list("detailed", "systematic")

In [4]:
for (dataset in datasets) {
    # read data
    feature.dt <- fread(paste("../../training_data/", dataset, "/features.csv", sep = ""))
    target.dt  <- fread(paste("../../training_data/", dataset, "/target.csv", sep = ""))
    folds.dt   <- fread(paste("../../training_data/", dataset, "/folds.csv", sep = ""))
    
    # filter censored intervals
    target.dt  <- target.dt[!(min.log.lambda == -Inf & max.log.lambda == Inf)]
    feature.dt <- feature.dt[sequenceID %in% target.dt$sequenceID]
    folds.dt   <- folds.dt[sequenceID %in% target.dt$sequenceID]

    # combine
    feature.dt.all <- cbind(feature.dt, folds.dt[, "fold"])
    target.dt.all  <- cbind(target.dt, folds.dt[, "fold"])
    
    # get train and test data
    feature.mat.train <- as.matrix(feature.dt.all[feature.dt.all$fold == "short", -c('sequenceID', 'fold')])
    feature.mat.test  <- as.matrix(feature.dt.all[feature.dt.all$fold == "long", -c('sequenceID', 'fold')])
    target.mat.train  <- as.matrix(target.dt.all[target.dt.all$fold == "short", c("min.log.lambda", "max.log.lambda")])
    
    # train model
    fit <- IntervalRegressionCV(feature.mat = feature.mat.train, target.mat = target.mat.train)
    
    # get prediction from test set
    target.mat.pred <- fit$predict(feature.mat.test)
    colnames(target.mat.pred) <- "llda"

    # save prediction to csv
    prediction <- data.frame(as.matrix(feature.dt.all[feature.dt.all$fold == "long", "sequenceID"]), target.mat.pred)
    write.csv(prediction, file = paste("predictions/", paste(dataset, "csv", sep = "."), sep=''), row.names = FALSE)
}